In [1]:
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import re
import datetime

In [2]:
#获取202207往后12个月的月份list（为了后续读取文件）
month_date_list = list(pd.period_range('2022-07',periods=12,freq='M'))
monthlist = list(map(lambda x : str(x)[:4] + str(x)[-2:] , month_date_list))

In [3]:
#读取全部热搜文件并合并
monthallnewslist = []
for month in tqdm(monthlist):
    news = pd.read_excel(month + '.xlsx')
    monthallnewslist.append(news.drop('序号',axis=1))
newsall = pd.concat(monthallnewslist)
newsall.sort_values('上榜时间',inplace=True)
newsalldf = newsall.set_index('上榜时间')
newsalldf.index = pd.to_datetime(newsalldf.index)

100%|██████████| 12/12 [00:10<00:00,  1.10it/s]


In [4]:
#删去广告
newsalldf = newsalldf[newsalldf['备注']!='广告位']

,热搜,最高排名,在榜时长（分钟）,热度,备注
上榜时间,,,,,
2022-07-01 00:00:10,RNG战胜OMG,47,20分,67801,NaN
2022-07-01 00:00:10,湖南日军性奴隶制度受害者雷金二逝世,21,10时4分,131081,NaN
2022-07-01 00:00:10,中国女排3比0波兰女排,20,7时17分,47838,NaN
2022-07-01 00:00:10,西瓜最清凉的吃法,19,4时59分,7128,NaN
2022-07-01 00:00:10,话梅刺客,6,9时24分,167708,NaN
...,...,...,...,...,...
2023-06-30 23:32:20,小学生平静分享保持情绪稳定的秘诀,46,6分,185990,NaN
2023-06-30 23:34:16,裸体男子在女浴室待8分钟称误入,23,25分,386627,NaN
2023-06-30 23:42:55,什么病人明明是我本人,39,16分,237910,NaN


In [5]:
#获取交易日
close = pd.read_feather('BasicFactor_Close.txt').set_index('time')
close.index = pd.to_datetime(close.index.astype(str))
tradeday = close['2022-07-01':].index

In [6]:
#构造交易日开盘时间，交易日停盘时间，前一个交易日开盘时间
start_tradeday = list(map(lambda x : x + datetime.timedelta(hours=9,minutes=15) , tradeday))
end_tradeday = list(map(lambda x : x + datetime.timedelta(hours=15) , tradeday))
previous_end_tradeday = end_tradeday.copy()
previous_end_tradeday.insert(0,pd.Timestamp('2022-06-30 15:00:00'))
previous_end_tradeday.pop()

Timestamp('2023-06-30 15:00:00')

In [7]:
#盘前news；盘中news，盘后news
preopen_timepair_list = list(zip(previous_end_tradeday,start_tradeday))
opening_timepair_list = list(zip(start_tradeday,end_tradeday))
all_timepair_list = list(zip(previous_end_tradeday,end_tradeday))

In [13]:
#获取三组热搜数据表
def getnewscut(newsdf,timepairlist):
    daynewslist = []
    for timepair in timepairlist:
        daynews = newsdf.truncate(before=timepair[0], after=timepair[1])
        day = timepair[1].date()
        daynews = pd.concat([daynews,pd.Series(day,name='交易日',index=daynews.index)],axis=1)
        daynewslist.append(daynews)
    finalnews = pd.concat(daynewslist)
    return finalnews

In [14]:
preopen_news = getnewscut(newsalldf,preopen_timepair_list)
opening_news = getnewscut(newsalldf,opening_timepair_list)
all_news = getnewscut(newsalldf,all_timepair_list)

In [15]:
preopen_news.to_csv('preopen_news.csv')
opening_news.to_csv('opening_news.csv')
all_news.to_csv('all_news.csv')